# JIT compilation

Julia code is JIT-compiled on a language level:

In [2]:
# import Pkg; Pkg.add("BenchmarkTools")

  Installing known registries into `C:\Users\sfrie\.julia`


     Cloning registry from "https://github.com/JuliaRegistries/General.git"


       Added registry `General` to `C:\Users\sfrie\.julia\registries\General`


   Resolving package versions...

   Installed JSON ─────────── v0.21.2
   Installed Parsers ──────── v2.1.2
   Installed BenchmarkTools ─ v1.2.0


...


Precompiling project...


  ✓ Parsers


  ✓ JSON


  ✓ BenchmarkTools


  3 dependencies successfully precompiled in 9 seconds


In [1]:
using BenchmarkTools

`:` is used to create ranges, both integer and floating point. Note, that Julia uses `1`-based indexing and hence right edge is included:

In [2]:
N = 100000
# : is used to create ranges
a = collect(1.:N)
typeof(a)

Vector{Float64} (alias for Array{Float64, 1})

In [3]:
a[1:5] #Julia is a 1 based language, not zero like in python

5-element Array{Float64,1}:
 1.0
 2.0
 3.0
 4.0
 5.0

In [4]:
a[end-4:end]

5-element Array{Float64,1}:
  99996.0
  99997.0
  99998.0
  99999.0
 100000.0

In [5]:
@btime sum(a) #@ sign operations just represents a macro for a different code

  12.001 μs (1 allocation: 16 bytes)


5.00005e9

In [6]:
@btime reduce(+, a)

  12.154 μs (1 allocation: 16 bytes)


5.00005e9

In [7]:
function sum_array(a)
    result = 0.

    for el in a
        result += el
    end
    return result
end

sum_array (generic function with 1 method)

In [8]:
@assert sum_array(a) == sum(a)

In [9]:
@btime sum_array(a)

  89.134 μs (1 allocation: 16 bytes)


5.00005e9

You can always look inside the code on different levels:

In [10]:
@code_typed sum_array(a)

CodeInfo(
1 ── %1  = Base.bitcast(UInt64, 1)::UInt64
│    %2  = Base.bitcast(UInt64, 1)::UInt64
│    %3  = Base.sub_int(%1, %2)::UInt64
│    %4  = Base.arraylen(a)::Int64
│    %5  = Base.sle_int(0, %4)::Bool
│    %6  = Base.bitcast(UInt64, %4)::UInt64
│    %7  = Base.ult_int(%3, %6)::Bool
│    %8  = Base.and_int(%5, %7)::Bool
└───       goto #3 if not %8
2 ── %10 = Base.arrayref(false, a, 1)::Float64
│    %11 = Base.add_int(1, 1)::Int64
└───       goto #4
3 ──       goto #4
4 ┄─ %14 = φ (#2 => false, #3 => true)::Bool
│    %15 = φ (#2 => %10)::Float64
│    %16 = φ (#2 => %11)::Int64
└───       goto #5
5 ── %18 = Base.not_int(%14)::Bool
└───       goto #11 if not %18
6 ┄─ %20 = φ (#5 => 0.0, #10 => %23)::Float64
│    %21 = φ (#5 => %15, #10 => %37)::Float64
│    %22 = φ (#5 => %16, #10 => %38)::Int64
│    %23 = Base.add_float(%20, %21)::Float64
│    %24 = Base.bitcast(UInt64, %22)::UInt64
│    %25 = Base.bitcast(UInt64, 1)::UInt64
│    %26 = Base.sub_int(%24, %25)::UInt64
│    %27 = Bas

# Multiple dispatch

Custom types are as first-class citizens as built-in's are:

In julia, there are no classes, rather "structs". They represent types, which are used to dispatch execution of the functions

In [3]:
abstract type Geometry end #This creates a type hierarchy

mutable struct Point{N<:Number} <: Geometry #Struct point is a child of geometry. We are saying htat X and Y can contain any type, just that it has to be a number
    x::N
    y::N
end

Note, that `Point` is a *parametrized* type:

In [5]:
Point(1,2)

Point{Int64}(1, 2)

In [6]:
Point(1., 2.)

Point{Float64}(1.0, 2.0)

We can make it look nearly as numbers:

In [9]:
import Base: +, -, /, *

In [10]:
+(a::Point, b::Point) = Point(a.x + b.x, a.y + b.y) #Define a new method for addition. So whenever we want to add two Points, these two methods will be used.
-(a::Point, b::Point) = Point(a.x - b.x, a.y - b.y)
/(a::Point, f::Number) = Point(a.x / f, a.y / f)

/ (generic function with 107 methods)

In [11]:
Point(1., 2.) + Point(3, 4)

Point{Float64}(4.0, 6.0)

In [12]:
Point(1., 2.) - Point(3, 4)

Point{Float64}(-2.0, -2.0)

In [13]:
(Point(1., 2.) + Point(3, 4)) / 2

Point{Float64}(2.0, 3.0)

Or create array with it:

In [19]:
[Point(a, a+1) for a in 1:10]

10-element Array{Point{Int64},1}:
 Point{Int64}(1, 2)
 Point{Int64}(2, 3)
 Point{Int64}(3, 4)
 Point{Int64}(4, 5)
 Point{Int64}(5, 6)
 Point{Int64}(6, 7)
 Point{Int64}(7, 8)
 Point{Int64}(8, 9)
 Point{Int64}(9, 10)
 Point{Int64}(10, 11)

Generic programming works nicely:

In [14]:
using Statistics
mean([Point(a, a+1) for a in 1:10])

Point{Float64}(5.5, 6.5)

Function in Julia have *methods* (i.e. definitions for various type combinations):

In [15]:
methods(+)

# 191 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(c::Union{UInt16, UInt32, UInt8}, x::BigInt) in Base.GMP at gmp.jl:528
[3] +(c::Union{Int16, Int32, Int8}, x::BigInt) in Base.GMP at gmp.jl:534
[4] +(c::Union{UInt16, UInt32, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:376
[5] +(c::Union{Int16, Int32, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:384
[6] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:392
[7] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\Dates\src\periods.jl:362
[8] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.Tridiagonal) in LinearAlgebra at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\LinearAlgebra\src\tridiag.jl:733
[9] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.SymTridiagonal) in LinearAlgebra at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:157
[10] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.Diagonal) in LinearAlgebra at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:173
[11] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:193
[12] +(A::LinearAlgebra.Tridiagonal{var"#s814", V} where {var"#s814"<:Number, V<:AbstractVector{var"#s814"}}, B::LinearAlgebra.UniformScaling) in LinearAlgebra at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:226
[13] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.Hermitian) in LinearAlgebra at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\LinearAlgebra\src\symmetric.jl:469
[14] +(H::LinearAlgebra.Hermitian, D::LinearAlgebra.Diagonal{var"#s814", V} where {var"#s814"<:Real, V<:AbstractVector{var"#s814"}}) in LinearAlgebra at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\LinearAlgebra\src\diagonal.jl:169
[15] +(A::LinearAlgebra.Hermitian, J::LinearAlgebra.UniformScaling{var"#s814"} where var"#s814"<:Complex) in LinearAlgebra at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\LinearAlgebra\src\uniformscaling.jl:196
[16] +(A::LinearAlgebra.Hermitian{var"#s812", var"#s811"} where {var"#s812", var"#s811"<:(SparseArrays.AbstractSparseMatrix{Tv, Ti} where {Tv, Ti})}, B::SparseArrays.AbstractSparseMatrix{Tv, Ti} where {Tv, Ti}) in SparseArrays at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\SparseArrays\src\linalg.jl:15
[17] +(A::LinearAlgebra.Hermitian, B::SparseArrays.AbstractSparseMatrix{Tv, Ti} where {Tv, Ti}) in SparseArrays at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\SparseArrays\src\linalg.jl:18
[18] +(A::LinearAlgebra.Hermitian{var"#s802", var"#s801"} where {var"#s802", var"#s801"<:(SparseArrays.AbstractSparseMatrix{Tv, Ti} where {Tv, Ti})}, B::LinearAlgebra.Symmetric{var"#s800", var"#s799"} where {var"#s800"<:Real, var"#s799"<:(SparseArrays.AbstractSparseMatrix{Tv, Ti} where {Tv, Ti})}) in SparseArrays at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\SparseArrays\src\linalg.jl:26
[19] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.Symmetric{var"#s814", S} where {var"#s814"<:Real, S<:(AbstractMatrix{var"#s814"} where var"#s814"<:var"#s814")}) in LinearAlgebra at C:\Users\sfrie\AppData\Local\Programs\Julia-1.6.3\share\julia\stdlib\v1.6\LinearAlgebra\src\symmetric.jl:483
[20] +(A::LinearAlgebra.Hermitian{var"#s810", var"#s809"} where {var"#s810", var"#s809"<:(SparseArrays.AbstractSparseMatrix{Tv, Ti} where {Tv, Ti})}, B::LinearAlgebra.Symmetric{var"#s808", var"#s807"} where {var"#s808", var"#s807"<:(SparseArrays.AbstractSparseMatrix{Tv, Ti

# Performance

Julia is generally faster or at least on par with NumPy (even with Numba's JIT). Linear algebra and high-performance array are parts of the language:

In [16]:
N = 1000
a = randn(Float64, (N, N))
b = randn(Float64, (N, N));

Dot product:

In [17]:
@btime a'b

  37.142 ms (3 allocations: 7.63 MiB)


1000×1000 Matrix{Float64}:
  35.0719     2.55508   -23.5133   …  -16.4329     80.1428    -5.81725
  43.4447    60.1029     88.9469      -23.5711    -58.4321   -31.5478
 -24.9623    10.848     -33.1031      -18.9542    -50.087    -47.2288
 -32.0018    52.1243    -17.0516        8.0373    -32.8635   -22.902
  12.9622     0.705472  -29.656       -19.0333     15.9855    51.4057
 -53.8903    -0.832835   15.4248   …   37.6994      2.67829   49.2886
 -42.6571    40.271      10.0743      -48.007      31.3472   -33.8919
   5.75821  -34.667      56.1406       -4.71211     7.22203   23.2524
  14.0739    47.6863      7.59661      38.7782    -39.6542   -12.3448
 -18.0648    -9.49424     9.1994        4.97529    -7.96556  -30.4275
   ⋮                               ⋱                         
  17.394     -2.09662     9.55351      -0.841151  -28.3907    52.6158
  -4.87735   20.752     -18.9958      -50.8547     -2.17405  -93.1279
  11.8946   -19.9064     20.5875      -13.4281     29.76      36.0749
 

# Vectorization and fusion

Let's define a scalar function:

In [18]:
relu(a::Number) = max(0, a)

relu (generic function with 1 method)

And vectorize (i.e. *broadcast*) it:

In [19]:
@btime relu.(a)

  1.820 ms (5 allocations: 7.63 MiB)


1000×1000 Matrix{Float64}:
 0.0        0.0        0.0       0.0       …  0.0       0.181974   0.34207
 0.0        0.0        0.319158  0.633009     0.0       0.0        0.0
 0.0        0.0        0.0       0.0          0.0       0.735232   1.76079
 0.636602   0.039068   0.0       1.53936      0.0       0.316189   0.250761
 0.0        0.0        0.0       0.0          0.0       0.0961098  0.233341
 0.114362   0.0        0.0       0.0       …  0.640049  0.0        0.0
 0.0        0.470206   1.65335   0.0          0.0       0.0        0.0
 0.0        0.0        0.0       0.607818     0.0       0.0        1.02304
 0.0        0.0        0.0       0.0          0.013587  0.0        0.3967
 0.0        0.61623    0.0       0.906179     1.54052   0.0        0.0
 ⋮                                         ⋱                       
 0.0        1.42016    0.0       0.0          0.328852  0.94829    0.0
 0.0        0.0        0.0       0.561237     0.804022  0.0        0.0
 0.173844   0.0378839  0.611

Of course, we do not need a separate functions for this, cause broadcasting works as expected directly:

In [26]:
max.(0, a)

1000×1000 Array{Float64,2}:
 0.0        0.0       0.760295   0.417175  …  1.29851     0.576061  0.0
 0.0        0.0       0.815669   0.0          0.0         0.956743  1.89822
 0.0        0.0       0.0        0.445039     2.07947     0.0       0.0
 2.62331    0.781411  0.0        0.695569     0.0         0.895726  0.521313
 0.220799   0.0       0.171349   0.169255     0.0         0.721496  0.0
 0.0        0.420295  0.0        0.0       …  0.551026    0.0       0.0
 0.0        0.0       0.529636   0.459494     0.0         0.469946  0.0
 0.934899   2.34038   0.0        0.0          0.0         1.47002   0.0821255
 0.0        0.584367  0.290647   0.640149     1.61115     0.0       0.0
 2.04438    0.219168  0.0181634  0.0          0.256746    0.0       0.123959
 0.0        0.0       0.686056   0.0       …  1.09091     0.0       0.64629
 0.323789   0.097196  1.24745    0.0          0.0         0.348634  0.0
 0.0        1.40279   3.26789    1.75513      0.0         0.237101  0.479302
 ⋮     

This may not sound cool, unless you broadcast and *fuse* (which is generally not possible in Python):

In [27]:
@btime relu.(relu.(a))

  639.813 μs (7 allocations: 7.63 MiB)


1000×1000 Array{Float64,2}:
 0.0        0.0       0.760295   0.417175  …  1.29851     0.576061  0.0
 0.0        0.0       0.815669   0.0          0.0         0.956743  1.89822
 0.0        0.0       0.0        0.445039     2.07947     0.0       0.0
 2.62331    0.781411  0.0        0.695569     0.0         0.895726  0.521313
 0.220799   0.0       0.171349   0.169255     0.0         0.721496  0.0
 0.0        0.420295  0.0        0.0       …  0.551026    0.0       0.0
 0.0        0.0       0.529636   0.459494     0.0         0.469946  0.0
 0.934899   2.34038   0.0        0.0          0.0         1.47002   0.0821255
 0.0        0.584367  0.290647   0.640149     1.61115     0.0       0.0
 2.04438    0.219168  0.0181634  0.0          0.256746    0.0       0.123959
 0.0        0.0       0.686056   0.0       …  1.09091     0.0       0.64629
 0.323789   0.097196  1.24745    0.0          0.0         0.348634  0.0
 0.0        1.40279   3.26789    1.75513      0.0         0.237101  0.479302
 ⋮     

# Custom types

Let's create a type for one-hot encoded matrix.

In [28]:
import Base: size, getindex

In [29]:
mutable struct OneHot <: AbstractArray{AbstractFloat, 2}
    nclasses::Int64
    classes::Array{Int64, 1}

    function OneHot(ncls, cls)
        ncls < maximum(cls) ? throw(ErrorException("actual number of classes $(maximum(cls)) > $ncls")) : new(ncls, cls)
    end
end

size(o::OneHot) = (length(o.classes), o.nclasses)

function Base.getindex(o::OneHot, i)
    o.classes[i]
end

function Base.getindex(o::OneHot, i, j)
    Float64(o.classes[i] == j)
end


In [30]:
OneHot(2, [1, 3, 1])

LoadError: actual number of classes 3 > 2

In [31]:
o = OneHot(2, [1, 2, 1])

3×2 OneHot:
 1.0  0.0
 0.0  1.0
 1.0  0.0

In [32]:
o[2]

2

In [33]:
A = [[1, 2, 3] [4, 5, 6] [7, 8, 9]]
v = [1, 2, 3]

3-element Array{Int64,1}:
 1
 2
 3

In [34]:
A

3×3 Array{Int64,2}:
 1  4  7
 2  5  8
 3  6  9

In [35]:
A * o

3×2 Array{Any,2}:
  8.0  4.0
 10.0  5.0
 12.0  6.0

In [36]:
@btime A * o

  97.277 ns (7 allocations: 224 bytes)


3×2 Array{Any,2}:
  8.0  4.0
 10.0  5.0
 12.0  6.0

In [37]:
o_ar = [[1., 0, 1] [0, 1, 0]]

3×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0
 1.0  0.0

In [38]:
A * o_ar

3×2 Array{Float64,2}:
  8.0  4.0
 10.0  5.0
 12.0  6.0

In [39]:
@btime A * o_ar

  171.811 ns (5 allocations: 352 bytes)


3×2 Array{Float64,2}:
  8.0  4.0
 10.0  5.0
 12.0  6.0